# For what?

Description and Summary : https://www.evernote.com/l/Afl3pVV4oPVGJZ4bjAOqYAF42LDDHDtuIYw/ </bs>

Raw resource : https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview

# Install Library
- 만약 library가 importing 되지 않으시면 아래의 코드를 실행하시면 됩니다.
- plotly인 경우 가장 최신 버전을 설치하시면 plotly함수에서 가끔 오류가 발생합니다. 그러므로 3.10.0 version을 설치하실 때 가장 안정적인 동적 시각화를 하실 수 있습니다.

In [1]:
# !pip install swifter
# !pip install yellowbrick
# !pip install plotly==3.10.0

# Import library

In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

from statsmodels.graphics.gofplots import qqplot
from scipy.stats import kurtosis, skew
from yellowbrick.target import FeatureCorrelation,ClassBalance
import swifter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy.special import boxcox1p
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시
from matplotlib import font_manager, rc

## 한글 깨짐 방지 목적 
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)

## For Mac
# rc('font', family='/Library/Fonts/AppleGothic.ttf')

import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# import data
- sample_submission.csv file은 EDA 작업에서 당장 필요하지 않은 파일이므로 생략하였습니다.

In [3]:
item_cate = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')
train = pd.read_csv('sales_train_v2.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')

https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data 와 실제 데이터의 shape를 대조해본 결과 pandas를 통한 data importing part에서 누락치는 존재하지 않습니다.

In [4]:
for i in [item_cate,items,train,shops,test]:
    print(i.shape)
    print()

(84, 2)

(22170, 3)

(2935849, 6)

(60, 2)

(214200, 3)



# Individual EDA & Merge Table

{'items.csv','item_categories.csv'}에 대한 데이터 설명을 살펴보면 다음과 같습니다. 
- items.csv - supplemental information about the items/products.
- item_categories.csv  - supplemental information about the items categories.

{'items.csv','item_categories.csv'}에 대한 결측치는 존재하지 않습니다.

In [5]:
def checknull(data):
    return data.isnull().sum().to_frame()

'item_category_name' 변수를 기준으로 추론할 때, 러시아 회사에서 제공된 데이터임을 알 수 있습니다.

In [6]:
item_cate.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [7]:
checknull(item_cate)

,0
item_category_name,0
item_category_id,0


In [8]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
checknull(items)

,0
item_name,0
item_id,0
item_category_id,0


공통 인자인 'item_category_id'를 통해 Inner join을 진행합니다.

In [10]:
items_merge = pd.merge(items,
                       item_cate,
                       how = 'inner',
                       on = 'item_category_id')

In [11]:
items_merge.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Кино - DVD


아이템에 대한 항목('item_category_name')의 개수는 84개입니다.

In [12]:
print(items_merge.shape)
print(len(np.unique(items_merge['item_name'])))
print(len(np.unique(items_merge['item_category_name'])))

(22170, 4)
22170
84


In [13]:
def pichart(data,cri):    
    o1 = data[cri].value_counts().index
    o2 = data[cri].value_counts().values

    trace = go.Pie(labels = o1, values = o2)
    iplot([trace])

조인된 items_merge table에서 'item_category_name'을 기준으로 상위 5개 항목을 살펴보면 다음과 같습니다. 
- 'Кино - DVD' : 22.7%
- 'Музыка - CD локального производства' : 10.7%
- 'Кино - Blu-Ray' : 8.03%
- 'Игры PC - Цифра' : 5.07%
- 'Музыка - Винил' : 3.56%

전체 아이템의 50% 이상이 상위 다섯가지 항목에 귀속됩니다. 

In [14]:
pichart(items_merge,'item_category_name')

{'items.csv','item_categories.csv'}을 제외한 {train,test,shops} 테이블의 결측치는 조사 결과 존재하지 않습니다.

In [15]:
print(checknull(train).sum()[0])
print(checknull(test).sum()[0])
print(checknull(shops).sum()[0])

0
0
0


train data의 data type을 살펴 보았습니다. 'date'는 datetime형태의 데이터 타입이어야 하는데 'object'임이 확인되었습니다. 데이터 타입을 적절하게 변형시킵니다.

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [17]:
def add_time_features(data,x):
    data[x] = pd.to_datetime(data[x],format = '%d.%m.%Y')
    data['year'] = data[x].dt.year
    data['year_month'] = data[x].dt.to_period('M')
    data['month'] = data[x].dt.month
    data['quarter'] = data[x].dt.quarter
    data['weekday'] = data[x].dt.weekday
    data['weekday_name'] = data[x].dt.weekday_name
    data['day'] = data[x].dt.day
    return data

In [18]:
train = add_time_features(train,'date')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,year_month,month,quarter,weekday,weekday_name,day
0,2013-01-02,0,59,22154,999.00,1.0,2013,2013-01,1,1,2,Wednesday,2
1,2013-01-03,0,25,2552,899.00,1.0,2013,2013-01,1,1,3,Thursday,3
2,2013-01-05,0,25,2552,899.00,-1.0,2013,2013-01,1,1,5,Saturday,5
3,2013-01-06,0,25,2554,1709.05,1.0,2013,2013-01,1,1,6,Sunday,6
4,2013-01-15,0,25,2555,1099.00,1.0,2013,2013-01,1,1,1,Tuesday,15


train table에서 'date'를 살펴 보았을 때, 가장 첫번째 날은 '2013-01-01'이고 가장 마지막 날은 '2015-10-31'입니다.

In [19]:
print('First date in train set',np.unique(train['date'])[0])
print('Last date in train set',np.unique(train['date'])[-1])

First date in train set 2013-01-01T00:00:00.000000000
Last date in train set 2015-10-31T00:00:00.000000000


가장 거래가 활발한 날은 '2013-12-28'(9434회)이고 '2015-05-12'(1123회)에 가장 거래량이 적었습니다. 

In [20]:
vd = pd.DataFrame(train['date'].value_counts())
print(vd.index[0],vd.values[0][0])
print(vd.index[-1],vd.values[-1][0])

2013-12-28 00:00:00 9434
2015-05-12 00:00:00 1123


date를 '2013-01-01'부터 '2015-10-31'까지 순서대로 정렬한 후 해당날의 거래량을 distribution plot을 그려 볼 때, 거의 대부분의 값들이 일정하지만 특정날에 엄청난 거래가 발생함을 파악할 수 있습니다.

In [21]:
vd2 = vd.sort_index()
x = vd2['date'].values
group_labels = ['dist plot']
fig = ff.create_distplot([x],group_labels)
iplot(fig)

- 머신러닝의 문제 정의는 데이터의 성격에 따라 크게 네 가지로 구분할 수 있습니다.
  - 1) Supervised Learning 
  - 2) Unsupervised Learning
  - 3) Semi-Supervised Learning
  - 4) Reinforcement Learning
 
- 우리가 지금 머신러닝 모델을 만드려는 목적을 다시 한번 상기해 보았습니다. 'train set'을 기반으로 Test set에서 각 상점에서 팔린 상품들의 총량을 예측하는 것입니다.
- 다시말해서 'item_cnt_day'의 월 기준 총양을 예측하는 Supervised Learing 문제입니다
- 즉 명확한 반응변수가 존재하는 Regression 문제임을 파악할 수 있습니다.

Merge {train,items_merge(items,cate)} and {test,items_merge(items,cate)} tables

In [22]:
print('not merged train data shape ', train.shape)
print('not merged test data shape',test.shape)

not merged train data shape  (2935849, 13)
not merged test data shape (214200, 3)


In [23]:
train = pd.merge(train,
                 items_merge,
                 how = 'inner',
                 on = 'item_id')

In [24]:
test = pd.merge(test,
                items_merge,
                how = 'inner',
                on = 'item_id')

In [25]:
print('merged train data set shape : ',train.shape)
print('merged test data set shape : ',test.shape)

merged train data set shape :  (2935849, 16)
merged test data set shape :  (214200, 6)


In [26]:
print('shops data set has columns : ', shops.columns)

shops data set has columns :  Index(['shop_name', 'shop_id'], dtype='object')


제공된 상점들의 개수는 총 60개입니다.

In [27]:
print('shape of shops data set : ', shops.shape)
print('unique value of shops name : ',len(np.unique(shops['shop_name'])))

shape of shops data set :  (60, 2)
unique value of shops name :  60


Merge {train,items_merge(items,cate),shops} and {test,items_merge(items,cate),shops} tables

In [28]:
print('not merged train data shape ', train.shape)
print('not merged test data shape',test.shape)

not merged train data shape  (2935849, 16)
not merged test data shape (214200, 6)


In [29]:
train = pd.merge(train,
                 shops,
                 how = 'inner',
                 on = 'shop_id')

In [30]:
test = pd.merge(test,
                shops,
                how = 'inner',
                on = 'shop_id')

In [31]:
print('merged train data shape ', train.shape)
print('merged test data shape',test.shape)

merged train data shape  (2935849, 17)
merged test data shape (214200, 7)


merging이 완료된 테이블을 살펴보면 다음의 특성을 파악할 수 있습니다.
- 'item_category_id' 측면에서 살펴볼 때 train set은 84개이지만 test set은 62개임이 확인됩니다.['item_category_name' 상동]
- 'item_id' 측면에서 살펴볼 때 train set은 21807개이지만 test set은 5100개입니다.['item_name' 상동]
- 'shop_id' 측면에서 살펴볼 때 train set은 60개이지만 test set은 42개입니다.['shop_name' 상동]

In [32]:
items_merge.nunique(axis = 0)

item_name             22170
item_id               22170
item_category_id         84
item_category_name       84
dtype: int64

In [33]:
shops.nunique(axis = 0)

shop_name    60
shop_id      60
dtype: int64

In [34]:
train.nunique(axis = 0)

date                   1034
date_block_num           34
shop_id                  60
item_id               21807
item_price            19993
item_cnt_day            198
year                      3
year_month               34
month                    12
quarter                   4
weekday                   7
weekday_name              7
day                      31
item_name             21807
item_category_id         84
item_category_name       84
shop_name                60
dtype: int64

In [35]:
test.nunique(axis = 0)

ID                    214200
shop_id                   42
item_id                 5100
item_name               5100
item_category_id          62
item_category_name        62
shop_name                 42
dtype: int64

# Question1 : Segment를 큰 범주에서 작은 범주로 나눴을 때 EDA의 결과는 얼마만큼 다른가?

## Components of segment
- time('year', 'quarter', 'month' , 'day') 
- item('item_category_id', 'item_id')
- shops('shop_name')

## Object
- 'item_category_id', 'item_price'

## Actualization

train data를 기준으로 첫번째 Question을 푸는 것에는 많은 자원과 시간의 한계가 소요됩니다(train set 관측치 : 2935849). 그러므로 test data에 속하지 않는 상점과 아이템은 제외합니다.
- first criteria : shop_name 
- second criteria : item_category_id
- third criteria : item_id

In [36]:
print(train.shape)
train = train[train['shop_name'].isin(np.unique(test['shop_name']))]
print(train.shape)
train = train[train['item_category_id'].isin(np.unique(test['item_category_id']))]
print(train.shape)
train = train[train['item_id'].isin(np.unique(test['item_id']))]
print(train.shape)

(2935849, 17)
(2413246, 17)
(2389027, 17)
(1224439, 17)


In [37]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,year_month,month,quarter,weekday,weekday_name,day,item_name,item_category_id,item_category_name,shop_name
0,2013-01-02,0,59,22154,999.0,1.0,2013,2013-01,1,1,2,Wednesday,2,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
2,2013-01-26,0,59,2574,399.0,1.0,2013,2013-01,1,1,5,Saturday,26,DEL REY LANA Born To Die The Paradise Editio...,55,Музыка - CD локального производства,"Ярославль ТЦ ""Альтаир"""
3,2013-01-09,0,59,2574,399.0,1.0,2013,2013-01,1,1,2,Wednesday,9,DEL REY LANA Born To Die The Paradise Editio...,55,Музыка - CD локального производства,"Ярославль ТЦ ""Альтаир"""
4,2013-02-24,1,59,2574,399.0,1.0,2013,2013-02,2,1,6,Sunday,24,DEL REY LANA Born To Die The Paradise Editio...,55,Музыка - CD локального производства,"Ярославль ТЦ ""Альтаир"""
5,2013-03-29,2,59,2574,399.0,1.0,2013,2013-03,3,1,4,Friday,29,DEL REY LANA Born To Die The Paradise Editio...,55,Музыка - CD локального производства,"Ярославль ТЦ ""Альтаир"""


각 상점별로 상위 열개 카테고리 품목을 Pie plot으로 그리면 다음과 같습니다. 모든 상점의 카테고리 기준 순위권에 PC, DVD 항목이 들어가 있네요. 

In [38]:
#pd.DataFrame(train[train['shop_id'] == 59]['item_id'].value_counts())['item_id'] > 1
w = []
for i in np.unique(train['shop_name']):    
    ex1 = pd.DataFrame(train[train['shop_name'] == i]['item_category_name'].value_counts())
    ex1['ratio'] = ex1['item_category_name'] / sum(ex1['item_category_name'])
    ex2 = ex1.iloc[0:9,:]
    ex2['name'] = i
    w.append(ex2)
    ob1 = ex2['item_category_name'].index.values
    ob2 = ex2['item_category_name'].values
    ob3 = ex2['ratio'].values

    trace = go.Pie(name = i,labels = ob1,values = ob2)
    iplot([trace])

In [39]:
exam1 = pd.concat(w[0:])
exam1

,item_category_name,ratio,name
Игры PC - Стандартные издания,2551,0.164878,"Адыгея ТЦ ""Мега"""
Игры - PS3,2081,0.134501,"Адыгея ТЦ ""Мега"""
Игры - XBOX 360,1735,0.112138,"Адыгея ТЦ ""Мега"""
Игры - PS4,1454,0.093976,"Адыгея ТЦ ""Мега"""
Кино - DVD,927,0.059915,"Адыгея ТЦ ""Мега"""
...,...,...,...
Игры - XBOX 360,1393,0.064235,"Ярославль ТЦ ""Альтаир"""
Игры PC - Дополнительные издания,1030,0.047496,"Ярославль ТЦ ""Альтаир"""
Игры - PS4,865,0.039887,"Ярославль ТЦ ""Альтаир"""
Кино - Blu-Ray,829,0.038227,"Ярославль ТЦ ""Альтаир"""


In [40]:
k = []
for i in range(0,len(exam1),9):
    w2 = exam1.iloc[i:i+1,:]
    k.append(w2)
k2 = pd.concat(k[0:])
k2

,item_category_name,ratio,name
Игры PC - Стандартные издания,2551,0.164878,"Адыгея ТЦ ""Мега"""
Игры PC - Стандартные издания,2381,0.163183,"Балашиха ТРК ""Октябрь-Киномир"""
Игры PC - Стандартные издания,4244,0.211460,"Волжский ТЦ ""Волга Молл"""
Игры PC - Стандартные издания,3584,0.170310,"Вологда ТРЦ ""Мармелад"""
Музыка - CD локального производства,6913,0.174933,"Воронеж (Плехановская, 13)"
Игры PC - Стандартные издания,5199,0.169024,"Воронеж ТРЦ ""Максимир"""
Игры PC - Стандартные издания,2170,0.194462,Жуковский ул. Чкалова 39м?
Книги - Методические материалы 1С,2411,0.125750,Интернет-магазин ЧС
Игры PC - Стандартные издания,3071,0.156708,"Казань ТЦ ""ПаркХаус"" II"
Игры PC - Стандартные издания,5873,0.192123,"Калуга ТРЦ ""XXI век"""


각 상점에서 가장 판매가 많이 되었던 카테고리에 속한 품목들과 연-월('year_month')을 기준으로 'item_cnt_day'의 합을 그래프로 표현하면 다음과 같습니다.

- 'item_cnt_day'가 발생한 시점(연-월 기준)이 하나인 상점도 존재하네요
- 또한 상점마다 집계한 것이므로 집계 결과의 관측치가 다름을 확인할 수 있습니다.
- 상점마다 'item_cnt_day'합의 분산과 패턴이 상이합니다.

In [41]:
for a1,a2 in enumerate(k2['name']):
    po = train[train['shop_name'] == a2]
    cri = k2.index[a1]
    wanna = po[po['item_category_name'] == cri]
    d = wanna.groupby(['year_month'])['item_cnt_day'].sum().reset_index()
    t_d = go.Scatter(x=d.index,y=d['item_cnt_day'])
    layout = go.Layout(title = 'sum of Item_cnt_day in ' + a2 + '_year_month' ,
                       xaxis = dict(title = 'Date'),
                       yaxis = dict(title = 'item_cnt_day'))
    fig = go.Figure(data = [t_d], layout = layout)
    iplot(fig)

이번에는 시간 단위를 넓혀서 각 상점에서 가장 판매가 많이 되었던 카테고리에 속한 품목들과 연-분기('year & Quarter')을 기준으로 'item_cnt_day'의 합을 그래프로 표현하면 다음과 같습니다.

In [42]:
for a1,a2 in enumerate(k2['name']):
    po = train[train['shop_name'] == a2]
    cri = k2.index[a1]
    wanna = po[po['item_category_name'] == cri]
    d = wanna.groupby(['year','quarter'])['item_cnt_day'].sum().reset_index()
    t_d = go.Scatter(x=d.index,y=d['item_cnt_day'])
    layout = go.Layout(title = 'sum of Item_cnt_day in ' + a2 + '_Year-Quarter',
                       xaxis = dict(title = 'Date'),
                       yaxis = dict(title = 'item_cnt_day'))
    fig = go.Figure(data = [t_d], layout = layout)
    iplot(fig)

이번에는 시간 단위를 좁혀서 각 상점에서 가장 판매가 많이 되었던 카테고리에 속한 품목들과 연-월-주간('year & month & Weekday')을 기준으로 'item_cnt_day'의 합을 그래프로 표현하면 다음과 같습니다.

- 시간 단위를 좁혀서 'item_cnt_day'를 살펴볼 수록 추세를 파악하기 용이함을 파악할 수 있습니다.
- 특히 주간으로 살펴보니 한 카테고리이지만 계절성이 보이네요.

In [43]:
for a1,a2 in enumerate(k2['name']):
    po = train[train['shop_name'] == a2]
    cri = k2.index[a1]
    wanna = po[po['item_category_name'] == cri]
    d = wanna.groupby(['year_month','weekday'])['item_cnt_day'].sum().reset_index()
    t_d = go.Scatter(x=d.index,y=d['item_cnt_day'])
    layout = go.Layout(title = 'sum of Item_cnt_day in ' + a2 + '_Year-month-week',
                       xaxis = dict(title = 'Date'),
                       yaxis = dict(title = 'item_cnt_day'))
    fig = go.Figure(data = [t_d], layout = layout)
    iplot(fig)

아래 코드는 주피터 노트북의 Memory 때문에 주석 처리하였습니다.

bar chart를 통해 각 상점의 시간별({'Year','Quarter','month','weekday'}) 'item_cnt_day'의 총양을 시각화한 코드입니다.

In [44]:
## Bar Chart

# for i in np.unique(train['shop_name']):
#     ji1 = train[train['shop_name'] == i]     
#     ji3 = ji1.groupby(['month'])['item_cnt_day'].sum().reset_index()    
#     han1 = ji3['month'].values
#     han2 = ji3['item_cnt_day'].values
#     fig = go.Figure()
#     fig.add_trace(go.Bar(x = han1,
#                          y = han2,
#                          text = han2,
#                          textposition = 'auto'))    
#     iplot(fig)  

각 상점의 'item_cnt_day' 총합에 관한 Box Plot을 시간('month')을 기준으로 시각화한 코드입니다. Box plot을 기준으로 이상치에 해당되는 관측치들이 다수 포착됨을 확인할 수 있습니다.

In [45]:
# def box_visual(data,x,y):
#     trace = go.Box(
#         x = data[x],
#         y = data[y],
#         marker = {'color' : 'green'},
#         boxmean = 'sd',
#         showlegend = True)
    
#     data = [trace]
#     iplot(data)
    
# for i in np.unique(train['shop_name']):
#     ji1 = train[train['shop_name'] == i] 
#     box_visual(ji1,'month','item_cnt_day')

Merging과 Data Preprocessing이 일차적으로 완료된 train data를 csv file로 변환한다.

In [46]:
#train.to_csv('merged_train.csv',encoding = 'utf-8')